In [ ]:
!pip3 install numerapi
!pip3 install livelossplot
!pip3 install pydot
!apt install -y graphviz 

In [ ]:
import numerapi
example_public_id  = "XXX"
example_secret_key = "YYY"
napi               = numerapi.NumerAPI(example_public_id, example_secret_key)
SESSION            = 'bernie'
datasetname        = napi.download_current_dataset(unzip=True).replace('.zip', '')

In [ ]:
import pandas as pd

training        = pd.read_csv(datasetname+'/numerai_training_data.csv')
tournament      = pd.read_csv(datasetname+'/numerai_tournament_data.csv', header=0, index_col=None)
validation_data = tournament[tournament.data_type=='validation']

In [ ]:
tournament.head()

In [ ]:
print("""
    training shape: {}
    tournament shape: {}""".format(training.shape, tournament.shape))

In [ ]:
features = [f for f in list(training) if 'feature' in f]
X        = training[features].values
X        = X.reshape(X.shape[0], 1, X.shape[1])

all_sessions = ['bernie', 'charles', 'elizabeth', 'jordan', 'ken']
for s in all_sessions:
    globals()['Y_{}'.format(s)] = training['target_{}'.format(s)].values

Y = globals()['Y_'+SESSION]

validation_data = tournament[tournament.data_type=='validation']
valX            = validation_data[features].values
valX            = valX.reshape(valX.shape[0], 1, valX.shape[1])

for s in all_sessions:
    globals()['valY_{}'.format(s)] = validation_data['target_{}'.format(s)].values

valY = globals()['valY_'+SESSION]
    
testX = tournament[features].values
testX = testX.reshape(testX.shape[0], 1, testX.shape[1])
ids   = tournament['id']

In [ ]:
X.shape, Y_bernie.shape, valX.shape, valY_bernie.shape

# Create model

## feature sorting by information gain

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
regr = RandomForestRegressor(max_depth=7, random_state=42)
regr.fit(X.reshape(X.shape[0], X.shape[2]), Y)

feature_gains = {}
for i, f in enumerate(regr.feature_importances_):
    feature_gains[i] = f

sorted_feature_indexes = list(map(lambda x: x[0], sorted(feature_gains.items(), key=lambda kv: kv[1], reverse=True)))

In [ ]:
X    = X[:,:,np.argsort(sorted_feature_indexes)].copy()
valX = valX[:,:,np.argsort(sorted_feature_indexes)].copy()

In [ ]:
import keras
from keras.models import Model
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers import Input, Conv1D, LSTM, AveragePooling1D, AlphaDropout
from livelossplot import PlotLossesKeras
from keras.callbacks import TensorBoard
from keras.initializers import lecun_normal
from AdamW import AdamW
from keras.callbacks import ReduceLROnPlateau

In [ ]:
timewindow  = X.shape[1]
numfeatures = X.shape[2]

inputs   = Input(shape=(timewindow, numfeatures), name='input')
x        = inputs

x  = Dense(128, activation="relu", name="encoderlayer1", kernel_initializer=lecun_normal())(x)
x  = Dense(64,  activation="relu", name="encoderlayer2", kernel_initializer=lecun_normal())(x)

x  = Dense(32,  activation="relu", name="encoder", kernel_initializer=lecun_normal())(x)

x  = Dense(64,  activation="relu", name="decoderlayer1", kernel_initializer=lecun_normal())(x)
x  = Dense(128, activation="relu", name="decoderlayer2", kernel_initializer=lecun_normal())(x)
x  = Dense(numfeatures, activation='sigmoid', name="decoder", kernel_initializer=lecun_normal())(x)


aemodel = Model(inputs=inputs, outputs=x)
aemodel.summary()

from keras.utils import plot_model
plot_model(aemodel, to_file='model.png')
from IPython.display import Image
Image(url= "model.png")

In [ ]:
batch_size = 1024
epochs     = 100

b, B, T = batch_size, X.shape[0], epochs
wd = 0.005 * (b/B/T)**0.5

aemodel.compile(loss='mean_squared_error',  optimizer=AdamW(weight_decay=wd), metrics=['mae'])

In [ ]:
ae_histories = []
for i in range(5):
    noise_factor = 0.00001
    noisyX = X + noise_factor * np.random.normal(loc=0.0, scale=1, size=X.shape)
    noisyvalX = valX + noise_factor * np.random.normal(loc=0.0, scale=1, size=valX.shape)

    ae_histories.append(
        aemodel.fit(noisyX, X, batch_size=batch_size, epochs=epochs, shuffle=True, 
                    callbacks=[PlotLossesKeras()], 
                    validation_data=(noisyvalX, valX)))
    
ae_histories.append(
    aemodel.fit(X, X, batch_size=batch_size, epochs=epochs, shuffle=False, 
                callbacks=[PlotLossesKeras()], 
                validation_data=(valX, valX)))

In [ ]:
train_loss, val_loss = [], []
for h in ae_histories:
    train_loss += h.history['loss']
    val_loss   += h.history['val_loss']
plt.plot(train_loss)
plt.plot(val_loss)
plt.show()

In [ ]:
compressed_model = Model(inputs=aemodel.input, outputs=aemodel.get_layer("encoder").output)
compressed_X     = compressed_model.predict(X)                                                                                                             
compressed_valX  = compressed_model.predict(valX)
compressed_testX = compressed_model.predict(testX)

In [ ]:
timewindow  = compressed_X.shape[1]
numfeatures = compressed_X.shape[2]

inputs   = Input(shape=(timewindow, numfeatures))
x        = inputs

x  = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu', name='convfeatures1')(x)
x  = AveragePooling1D(pool_size=2, padding='same')(x)

x  = Dense(256, activation='selu', kernel_initializer=lecun_normal())(x)
x  = AlphaDropout(0.1)(x)
x  = Dense(128, activation='selu', kernel_initializer=lecun_normal())(x)
x  = AlphaDropout(0.1)(x)
x  = Dense(64, activation='selu', kernel_initializer=lecun_normal())(x)
x  = AlphaDropout(0.1)(x)

x  = Flatten()(x)
x  = Dense(1, activation='sigmoid')(x)

classifier = Model(inputs=inputs, outputs=x)
classifier.summary()

from keras.utils import plot_model
plot_model(classifier, to_file='classifier.png')
from IPython.display import Image
Image(url= "classifier.png")

In [ ]:
batch_size = 1024
epochs     = 100

b, B, T = batch_size, X.shape[0], epochs
wd = 0.005 * (b/B/T)**0.5

classifier.compile(loss='binary_crossentropy', 
                          optimizer=AdamW(weight_decay=wd),
                          metrics=['accuracy'])

REDUCE_LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)

!rm -rf /tmp/classifier-c-s-g
history = classifier.fit(compressed_X, Y, batch_size=batch_size, epochs=epochs, shuffle=True, 
                             callbacks=[PlotLossesKeras(), REDUCE_LR], 
                             validation_data=(compressed_valX, valY))

In [ ]:
pred = classifier.predict(compressed_testX)
pred = pred.reshape(pred.shape[0])
print ("min: {}, max: {}".format(pred.min(), pred.max()))
results_df  = pd.DataFrame(data={'probability_bernie': pred})
joined      = pd.DataFrame(ids).join(results_df)
joined.to_csv('predictions_{}.csv'.format(SESSION), index=False)

In [ ]:
submission_id = napi.upload_predictions("predictions_bernie.csv")

In [ ]:
napi.submission_status()